In [54]:
import sys
import numpy as np
# import scikitlearn
import matplotlib
import pandas as pd
import glob

# ML Magic:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

# Local includes:
sys.path.append("../../src/")
import therm_params as tp
from power_model import (
    leakagePower,
    peripheral_power,
    board_power
)


In [40]:
# blackscholes_files = glob.glob("../blackscholes_benchmarking/*.csv")
blackscholes_files = ["../blackscholes_benchmarking/userspace_ondemand_blackscholes.csv"]
bodytrack_files = glob.glob("../bodytrack_benchmarking/*.csv")
random_data = ["../random_test.csv"]
files = random_data

In [45]:
df_list = []

def leak_row_big(row):
    return leakagePower(tp.c1, tp.c2, tp.Igate, row.volts_big_cluster, \
            np.max( [row.temp4, row.temp5, row.temp6, row.temp7] ) + 273.15)
    
def leak_row_little(row):
    return leakagePower(tp.c1, tp.c2, tp.Igate, row.volts_little_cluster, \
            np.mean( [row.temp4, row.temp5, row.temp6, row.temp7, row.temp_gpu] ) + 273.15)

def leak_row_gpu(row):
    return leakagePower(tp.c1, tp.c2, tp.Igate, row.volts_gpu, row.temp_gpu + 273.15 )

def leak_row_mem(row):
    return leakagePower(tp.c1, tp.c2, tp.Igate, row.volts_mem, \
            np.max( [row.temp4, row.temp5, row.temp6, row.temp7] ) + 273.15)

for file in files:
    df = pd.read_csv(file, index_col=False, header=0, sep='\t')
    df_list.append(df)

# for file in bodytrack_files:
#     df = pd.read_csv(file, index_col=False, header=0, sep='\t')
#     df_list.append(df)

data = pd.concat(df_list, ignore_index=True)
# Drop all measurements where watts <= 0:
data = data.loc[data.watts > 0]
# fill in the potentially missing voltage values
data["volts_big_cluster"] = data.freq_big_cluster.map(lambda x: tp.big_f_to_v[float(x)/1000000000])
data["volts_little_cluster"] = data.freq_little_cluster.map(lambda x: tp.little_f_to_v[float(x)/1000000000])
# Replace with correct value for gpu and mem voltage:
data["volts_gpu"] = 1.0
data["volts_mem"] = 1.2
# Recompute the leakage power:
data['leak_big'] = data.apply(leak_row_big, axis=1)
data['leak_little'] = data.apply(leak_row_little, axis=1)
data['leak_gpu'] = data.apply(leak_row_gpu, axis=1)
data['leak_mem'] = data.apply(leak_row_mem, axis=1)
data['w_leak2'] = data.apply(lambda row: np.sum( [row.leak_big, row.leak_little, row.leak_gpu, row.leak_mem] ), axis=1)

# data.drop(["volts_big_cluster", "volts_little_cluster", "volts_mem", "volts_gpu"], axis=1, inplace=True)

data_filtered = pd.DataFrame()
data_filtered["usage_little"] = data[["usage_c0", "usage_c1", "usage_c2", "usage_c3"]].apply(func = np.sum, axis=1)
data_filtered["usage_big"] = data[["usage_c4", "usage_c5", "usage_c6", "usage_c7"]].apply(func = np.sum, axis=1)
data_filtered["max_temp_big"] = data[["temp4", "temp5", "temp6", "temp7"]].apply(func = np.max, axis=1)
data_filtered["avg_temp_big"] = data[["temp4", "temp5", "temp6", "temp7"]].apply(func = np.mean, axis=1)
data_filtered["gpu_temp"] = data.temp_gpu
data_filtered["vvf_b"] = data.freq_big_cluster.map(lambda x: tp.big_f_to_v[float(x)/1000000000]**2 * x)
data_filtered["vvf_l"] = data.freq_little_cluster.map(lambda x: tp.little_f_to_v[float(x)/1000000000]**2 * x)
data_filtered["p_dyn"] = data.apply(lambda row: row.watts - row.w_leak2 - peripheral_power, axis=1)
data_filtered["aVVFb"] = data_filtered.apply(lambda row: row.usage_big * row.vvf_b, axis=1)
data_filtered["aVVFl"] = data_filtered.apply(lambda row: row.usage_little * row.vvf_l, axis=1)
data_filtered["gpu_mem"] = 1

In [46]:
data_filtered.head(10)

,usage_little,usage_big,max_temp_big,avg_temp_big,gpu_temp,vvf_b,vvf_l,p_dyn,aVVFb,aVVFl,gpu_mem
1,0.000000,0.000000,57.0,55.25,53.0,486000000.0,1.155625e+09,0.819187,0.000000e+00,0.000000e+00,1
2,0.000000,0.000000,57.0,55.00,53.0,486000000.0,1.155625e+09,0.819719,0.000000e+00,0.000000e+00,1
3,0.047619,0.050000,57.0,55.00,53.0,486000000.0,1.155625e+09,0.518719,2.430000e+07,5.502976e+07,1
4,0.000000,0.050000,56.0,54.75,53.0,486000000.0,1.155625e+09,0.516676,2.430000e+07,0.000000e+00,1
5,0.000000,0.047619,56.0,54.75,53.0,486000000.0,1.155625e+09,0.527676,2.314286e+07,0.000000e+00,1
6,0.000000,0.050000,56.0,54.75,53.0,486000000.0,1.155625e+09,0.527676,2.430000e+07,0.000000e+00,1
7,0.000000,0.000000,56.0,54.75,53.0,486000000.0,1.155625e+09,0.524676,0.000000e+00,0.000000e+00,1
8,0.000000,0.050000,56.0,54.75,53.0,486000000.0,1.155625e+09,0.516676,2.430000e+07,0.000000e+00,1
9,0.000000,0.000000,56.0,54.75,53.0,486000000.0,1.155625e+09,0.512676,0.000000e+00,0.000000e+00,1
10,0.000000,0.047619,56.0,54.75,53.0,486000000.0,1.155625e+09,0.512676,2.314286e+07,0.000000e+00,1


In [58]:
X = data_filtered.loc[:, ["aVVFb", "vvf_b", "aVVFl", "vvf_l", "gpu_mem"]]
y = data_filtered.p_dyn

# Estimate C values for the model:
DPM = Ridge()
# # Create training and test sets:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42) 
DPM = DPM.fit(X_train,y_train)
P = DPM.predict(X_test)

In [ ]:
P
